# Applied Data Science Captstone Project

The Battle of Neighborhoods

## Introduction

People often find themselves in a situations where they have to move grand distances for a career, family obligations, or even a change of scenary. However, moving to a new city can be an extremely difficult process as they must leave the familiarity of their old home to an often unforgiving new city. One way to overcome this is to move into a neighborhood that most resembles their's back home.

Within this capstone, I aim to tackle this exact problem using a case study with Toronto and New York City. Both are massive cities with huge job markets, making them prime locations for job seekers to migrate both to and between. I will be analyzing the neighborhoods within each city and cross-comparing them to establish which neighborhoods in one city are most like those in the other. Finding similar neighborhoods will offer guidance to those wishing to move between them to find the neighborhood that most feels like home.

## Data Required and Utilization

The primary dataset that will be used is that recommended by the IBM learning team throughout the period of this capstone: Foursquare. This dataset alone allows for a substantial comparison between the neighborhoods of the two cities, however it only details the local attractions and places of business. Two neighborhoods could be extremely similar in this perspective, however they might be dramatically different in other aspects: most notably cost of housing. For New York, I will be use the housing data supplied by [Zillow](https://www.zillow.com/research/data/) as they have a massive real estate data collection for U.S. cities. However, they do not have this data readily available for Canadian cities. For Toronto, I will be using the data provided from the [Canada Mortgage and Housing Corporation](https://www.cmhc-schl.gc.ca/en/data-and-research/data-tables/price-quartiles-averages-absorbed-homeowner-condo-units). For an average house price, a single-family home will be used.

The data will be combined and aggregated by neighborhood. If a neighborhood is missing from either dataset, that neighborhood's house cost will be made equal to it's nearest neighbor (established through distance to neighborhood center). Once aggregated by neighborhood, several clustering algorithms (k-Means, Heirarchical, and DBSCN) and the final result will be neighborhoods in both Toronto and New York City classified into several groups of like characteristics

## Method

First things first, import all required libraries:

In [6]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# !conda install -c conda-forge lxml --yes
import lxml

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


Get all the neighborhoods of Toronto:

In [7]:
# Define the wikipedia URL
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Send a GET request to the wikipedia site
wiki_data = requests.get(wiki_url).text

# Get the first dataframe in the list, since it is the one of interest
wiki_contents = pd.read_html(wiki_data)[0]

# Drop all postal codes where a borough is not assigned and reassign column names
print("Number of postal codes imported: " + str(wiki_contents.shape[0]))
wiki_contents = wiki_contents[wiki_contents['Borough'] != 'Not assigned']
wiki_contents.columns = ['PostalCode', 'Borough', 'Neighborhood']
print("Number of postal codes with boroughs: " + str(wiki_contents.shape[0]))
wiki_contents.head()

Number of postal codes imported: 180
Number of postal codes with boroughs: 103


AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government